# Общие зависимости

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.utils import plot_model

2024-12-16 15:50:54.428423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 15:50:54.452059: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 15:50:52.189360: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-16 15:50:52.399027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734353452.617888   58659 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734353452.68

In [3]:
# Убедимся, что TensorFlow использует GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU доступен и будет использован для обучения.")
    except RuntimeError as e:
        print(f"Ошибка настройки памяти GPU: {e}")
else:
    print("GPU не обнаружен. Используется CPU.")

GPU не обнаружен. Используется CPU.


W0000 00:00:1734353464.589218   58659 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# Параметры датасета
DATASET_DIR = "/mnt/e/Agriculture-Vision-2021 2"  # Задайте путь к датасету
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
IMG_SIZE = 512  # Размер входного изображения
BATCH_SIZE = 16
NUM_CLASSES = 9  # Количество классов сегментации

# Функциональные классы

In [5]:
# Патчинг изображения
class PatchEmbedding(layers.Layer):
    def __init__(self, patch_size, embed_dim):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim

    def build(self, input_shape):
        self.projection = layers.Conv2D(
            filters=self.embed_dim,
            kernel_size=self.patch_size,
            strides=self.patch_size,
            padding="valid"
        )
        self.flatten = layers.Reshape((-1, self.embed_dim))

    def call(self, inputs):
        x = self.projection(inputs)
        x = self.flatten(x)
        return x

In [6]:
# Трансформерный блок
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout_rate=0.1):
        super().__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)

        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(dropout_rate),
            layers.Dense(embed_dim),
            layers.Dropout(dropout_rate),
        ])
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=False):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output)

        mlp_output = self.mlp(out1, training=training)
        return self.norm2(out1 + mlp_output)

# Настройка нейронной сети

In [7]:
# Модель ViT для семантической сегментации
class VisionTransformer(Model):
    def __init__(self, img_size, patch_size, num_classes, embed_dim, depth, num_heads, mlp_dim, dropout_rate=0.1):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        self.patch_embedding = PatchEmbedding(patch_size, embed_dim)
        self.position_embedding = self.add_weight(
            shape=(1, self.num_patches, embed_dim),
            initializer="random_normal",
            trainable=True
        )

        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, mlp_dim, dropout_rate) for _ in range(depth)
        ]

        self.decoder = tf.keras.Sequential([
            layers.Dense(embed_dim),
            layers.Reshape((img_size // patch_size, img_size // patch_size, embed_dim)),
            layers.Conv2DTranspose(filters=embed_dim // 2, kernel_size=2, strides=2, activation="relu"),
            layers.Conv2DTranspose(filters=embed_dim // 4, kernel_size=2, strides=2, activation="relu"),
            layers.Conv2D(num_classes, kernel_size=1, activation="softmax")
        ])

    def call(self, inputs):
        # Преобразование в патчи и добавление позиционной эмбеддинга
        x = self.patch_embedding(inputs)
        x += self.position_embedding

        # Пропуск через трансформерные блоки
        for block in self.transformer_blocks:
            x = block(x)

        # Декодер для восстановления пространственной размерности
        x = self.decoder(x)
        return x

In [8]:
# Параметры модели
PATCH_SIZE = 32 # Размер патча
EMBED_DIM = 512  # Увеличенная размерность эмбеддингов
DEPTH = 12  # Количество трансформерных блоков
NUM_HEADS = 16  # Голов для Multi-Head Attention
MLP_DIM = 1024  # Размерность скрытого слоя в MLP
DROPOUT_RATE = 0.1

In [9]:
# Создание модели
vit_segmentation_model = VisionTransformer(
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    num_classes=NUM_CLASSES,
    embed_dim=EMBED_DIM,
    depth=DEPTH,
    num_heads=NUM_HEADS,
    mlp_dim=MLP_DIM,
    dropout_rate=DROPOUT_RATE
)

# Компиляция модели
vit_segmentation_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
# Вывод структуры модели
vit_segmentation_model.build((None, IMG_SIZE, IMG_SIZE, 3))
vit_segmentation_model.summary()

# Сохранение схемы модели
# plot_model(vit_segmentation_model, to_file="vit_model_architecture.png", show_shapes=True)

/home/aboba/project/NeuralField/neuralfield/network/vit/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'vision_transformer', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "vision_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patch_embedding                 │ ?                      │   0 (unbuilt) │
│ (PatchEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_7             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_8             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_9             │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_10            │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_11            │ ?                      │   0 (unbuilt) │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_12 (Sequential)      │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,072 (512.00 KB)

 Trainable params: 131,072 (512.00 KB)

 Non-trainable params: 0 (0.00 B)

# Оценка обучения

In [11]:

# Функции для оценки
from sklearn.metrics import confusion_matrix, f1_score, jaccard_score


In [12]:
def evaluate_model(y_true, y_pred, num_classes):
    y_true_flat = y_true.flatten()
    y_pred_flat = np.argmax(y_pred, axis=-1).flatten()

    cm = confusion_matrix(y_true_flat, y_pred_flat, labels=range(num_classes))
    iou = jaccard_score(y_true_flat, y_pred_flat, average="macro")
    f1 = f1_score(y_true_flat, y_pred_flat, average="macro")

    mean_accuracy = np.diag(cm).sum() / cm.sum()

    return mean_accuracy, iou, f1

In [13]:
# Генерация гистограммы результатов
def plot_metrics(metrics, labels):
    plt.bar(labels, metrics, color=['blue', 'green', 'red'])
    plt.xlabel("Метрики")
    plt.ylabel("Значения")
    plt.title("Оценка модели")
    plt.show()

In [14]:
# График обучения
class TrainingPlotCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.history = {"loss": [], "accuracy": []}

    def on_epoch_end(self, epoch, logs=None):
        self.history["loss"].append(logs.get("loss"))
        self.history["accuracy"].append(logs.get("accuracy"))
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(self.history["loss"], label="Loss")
        plt.title("Training Loss")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.history["accuracy"], label="Accuracy")
        plt.title("Training Accuracy")
        plt.legend()

        plt.show()


In [15]:
# Функция для загрузки данных
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [16]:
def load_dataset(image_dir, mask_dir, img_size, count: int = 1000):
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    images = []
    masks = []

    for img_file, mask_file in zip(image_filenames, mask_filenames):
        count -= 1
        img = load_img(os.path.join(image_dir, img_file), target_size=(img_size, img_size))
        mask = load_img(os.path.join(mask_dir, mask_file), target_size=(img_size, img_size), color_mode="grayscale")
        images.append(img_to_array(img) / 255.0)
        masks.append(img_to_array(mask).astype("int"))
        if count == 0: 
            break


    return np.array(images), np.array(masks)


In [17]:
# Загрузка данных
train_images, train_masks = load_dataset(os.path.join(TRAIN_DIR, "images/rgb"), os.path.join(TRAIN_DIR, "masks"), IMG_SIZE, count = 200)
val_images, val_masks = load_dataset(os.path.join(VAL_DIR, "images"), os.path.join(VAL_DIR, "masks"), IMG_SIZE, count = 200)

IsADirectoryError: [Errno 21] Is a directory: '/mnt/e/Agriculture-Vision-2021 2/val/images/nir'

In [ ]:
# Обучение модели
history = vit_segmentation_model.fit(
    x=train_images,
    y=train_masks,
    validation_data=(val_images, val_masks),
    batch_size=BATCH_SIZE,
    epochs=50,
    callbacks=[TrainingPlotCallback()]
)

In [ ]:
# Пример использования метрик на валидационном наборе
val_predictions = vit_segmentation_model.predict(val_images)
metrics = evaluate_model(val_masks, val_predictions, NUM_CLASSES)
plot_metrics(metrics, ["Mean Accuracy", "IoU", "F1-Score"])

print("Обучение завершено. Схема модели сохранена в 'vit_model_architecture.png'")
